# Projeto 1 - Confiabilidade

## Autores

- D. H. Lelis - 12543822

## Orientações

Boa parte do código fonte foi organizado dentro de um módulo chamado `common`, disponibilizado juntamente ao notebook.
Alguns tópicos de atenção:

- **Dependências**
  - Neste notebook tem um bloco de código comentado que serve para instalar automaticamente as dependências, recomenda-se
    criar um ambiente virtual e instalá-las desta forma.
  - Caso crie uma venv, não esqueça de instalar o `jupyter` e/ou `ipykernel`.
  - **Importante:** *Nem todas as dependências utilizadas são comumente disponibilizadas
    em ambientes de execução como Google Colaboratory ou Anaconda.*
- **Configuração**
  - Caso seja necessário, é possível adequar algumas configurações para seu ambiente de execução (verificar `common/consts.py`
    e o bloco *"Configuração"*)
- **Simulações**
  - Por questões de performance, testamos desenvolver o simulador em algumas tecnologias. A versão mais completa e performática
    é a feita em *rust*, disponível em `/simulators/rust`, porém a versão em *Python*, principalmente usando *pypy*, consegue
    "quebrar o galho", estando disponível em `/simulators/python`.
  - Os resultados das simulações são armazenados em arquivos `.json` dentro da pasta `/datasets`. O notebook pode chamar os simuladores
    para gerar novas simulações caso não exista uma simulação com os parâmetros determinados. O simulador utilizado pode ser definido
    através das configurações.

### Dependências

In [1]:
# Instalação de dependências (descomentar linhas abaixo para instalar)
import sys

# !"{sys.executable}" -m pip install --upgrade pip
# !"{sys.executable}" -m pip install --upgrade pandas
# !"{sys.executable}" -m pip install --upgrade numpy
# !"{sys.executable}" -m pip install --upgrade scipy
# !"{sys.executable}" -m pip install --upgrade sympy
# !"{sys.executable}" -m pip install --upgrade matplotlib
# !"{sys.executable}" -m pip install --upgrade seaborn
# !"{sys.executable}" -m pip install --upgrade plotly

In [2]:
# Carregamento das dependências utilizadas

import sys
import os
import json
import math
import collections
import importlib

import numpy as np
import scipy as sc
import scipy.stats as st
import sympy as sp
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import List, Tuple, Callable, Dict

### Constantes

Verificar o arquivo `common/consts.py` para mais detalhes.

### Utilidades

Verificar os códigos em `common/` para mais detalhes.

## Enunciado

### Projeto 1 - Confiabilidade

Um sistema requer $n$ máquinas para funcionar. Elas quebram, a cada unidade de tempo (um minuto), com
probabilidade $p_0$. Para garantir o funcionamento confiável do sistema, $s$ máquinas adicionais são mantidas para
reposição. Quando uma máquina quebra, ela é substituída por uma em bom estado e enviada para reparo. O
tempo de reparo de cada máquina  ́e $t_R$ (fixo).

Deseja-se estudar a variável aleatória “tempo de colapso” do sistema, isto  ́e, o tempo $T$ que o sistema funciona
até dever ser parado porque não há $n$ máquinas disponíveis. Quanto vale $E(T)$? Qual é a distribuição de $T$?
Como depende a variável $T$ do número de máquinas reserva $s$? Mais em concreto, quantas máquinas adicionais
são necessárias para que $\text{Prob}\{T < 10000\}$ seja menor que 1%?

Faixas sugeridas de valores: $n = 10 − 100$, $p_0 = 0.001 − 0.01$, $s = 10 − 100$.

1. Quais das perguntas acima conseguem responder analiticamente? Nos casos que possa, qual é o resultado?
2. Os resultados do simulador são consistentes com as estimações analíticas no caso $\beta = 0$?
3. No simulador, é fácil inserir a amostragem de outras variáveis aleatórias. Estudar por exemplo a variável $Z$ que
podemos chamar “tempo de alerta”, definido como o tempo em que o número de máquinas em reparação atinge
80% do valor disponível $s$ (ou outra porcentagem a escolher). Como é a distribuição de probabilidade de $Z$?
4. São $Z$ e $T$ variáveis independentes? É possível predizer em alguma medida o tempo de colapso uma vez que foi
atingido o tempo de alerta? Se uma parada ordenada do sistema requer o tempo $t_R$ , pode sugerir uma estratégia
de manejo que “garanta” chegar a parada ordenada com 99% de probabilidade?
1. É lógico que as máquinas envelheçam. A probabilidade de cada máquina de falhar pode crescer linearmente com
o tempo, seguindo $p(k) = p_0 + \beta t_{f}(k)$ onde $k$ identifica a máquina e $t_f(k)$ o tempo que ela leva em funcionamento
desde o  ́ultimo reparo. No caso de $β \ne 0$  ́e impossível realizar cálculos analíticos sobre a variável $T$. Para isto,
desenvolver um pequeno código de simulação do processo. Esse caso é mais complexo que o original, porque é
necessário armazenar informação específica para cada máquina (o tempo que leva trabalhando). Mesmo assim,
trata-se apenas de uma adequada combinação de variáveis Bernoulli que o computador sabe simular.

## Parte 1 - Modelo Analítico e Validação da Simulação

### Uma simplificação do problema

Para encontrarmos uma solução analítica do problema, sentimos a necessidade de fazer algumas simplificações no problema, sendo elas: $t_R = \infty$ e $\beta = 0$. Dadas essas simplificações, conseguimos modelar nosso problema na forma de uma distribuição binomial negativa.

A representação mais comum da binomial negativa é dada por:

$$
p_1(t; s, p) = \begin{pmatrix}t + s - 1 \\ s -1\end{pmatrix} p^{s} (1 - p)^{t}
$$

Essa representação mede, dado um número fixo $s$ de sucessos desejados e uma probabilidade $p$, a probabilidade de obtermos $t$ falhas até atingir o último sucesso desejado.

Sabendo disso, podemos fazer algumas adaptações para melhor encaixá-la no problema. Primeiramente, ao invés de medirmos o número de falhas, vamos medir o número total de eventos necessários. Além disso, estamos interessados no ponto crítico do sistema, ou seja, quando atingimos $s_0 + 1$ falhas:

$$
p_2(t; s_0, p) = p_1(t-s_0-1; s_0 + 1, p) = \begin{pmatrix}t - 1 \\ s_0 \end{pmatrix} p^{s_0 + 1} (1 - p)^{t - s_0 - 1}
$$

Além disso, notemos que o problema considera um conjunto de $n$ máquinas em funcionamento de maneira simultânea. Podemos representar isso fazendo um agrupamento de $n$ eventos na nossa PMF, ficando da seguinte forma: ($k$ é o $k$-ésimo ciclo de execução)

$$
p_3(k; n, s_0, p) = \sum^{n}_{i = 1} p_2(n \cdot (k - 1) + i; s_0, p)
$$

Uma representação alternativa do formulado em $p_3$ é utilizando a CDF da binomial negativa, que, quando adequada ao nosso problema, pode ser representada da seguinte forma:

$$
I_{1-p}(s_0 + 1, k \cdot n - (s_0 + 1) + 1) = I_{1-p}(s_0 + 1, k \cdot n - s_0)
$$

*Obs.: $I$ corresponde à função beta incompleta regularizada* 

Além disso, o valor estimado pode ser representado da seguinte forma:

$$
E = \frac{(1 - p)(s_0 + 1) + p(s_0 + 1)}{pn} 
$$

#### A Variável $Z$ (Tempo de Alerta)

Dentro das mesmas simplificações, conseguimos mostrar que a modelagem da variável $Z$ seguirá a mesma distribuição, só que que com $s_0' = \lceil 0.8 \cdot s_0 \rceil$. Uma ideia semelhante se aplica ao tempo esperado de colapso.

### Comparando a solução analítica com a simulação



In [3]:
from common.models import tnbinom # Solução analítica
from common.simulation import load_or_generate_simulation, build_pmf # Simulação

In [4]:
def compare_model_and_sim(n: int, s: int, p: float):
    # Simulation
    sim_data = load_or_generate_simulation(1000000, n, p, s) # Load simulation data
    xs, sim_ys =  build_pmf(sim_data) # Build simulation PMF
    simulation_trace = go.Bar(name="Simulação", x=xs, y=sim_ys) # Create simulation plot

    # Model
    model_ys = tnbinom.pmf(xs, n, s, p) # Compute PMF for model at xs
    model_trace = go.Bar(name="Modelo", x=xs, y=model_ys) # Create model plot
    
    # Mean
    simulation_mean = np.mean(sim_data['results'])
    model_mean = tnbinom.mean(n, s, p)

    # Z Variable
    z_xs, sim_z_ys = build_pmf(sim_data, field='results_z')
    simulation_z_trace = go.Bar(name="Simulação Z", x=z_xs, y=sim_z_ys, visible='legendonly') # Create simulation plot for Z

    model_z_ys = tnbinom.pmf(z_xs, n, np.ceil(s * 0.8), p)
    model_z_trace = go.Bar(name="Modelo Z", x=z_xs, y=model_z_ys, visible='legendonly') # Create simulation plot for Z

    # Z-delta Variable
    delta_xs, sim_delta_ys = build_pmf(sim_data, field='results_delta')
    simulation_delta_trace = go.Bar(name="Simulação Delta", x=delta_xs, y=sim_delta_ys, visible='legendonly') 

    model_delta_ys = tnbinom.pmf(delta_xs, n, s - np.ceil(s * 0.8) - 1, p)
    model_delta_trace = go.Bar(name="Modelo Delta", x=delta_xs, y=model_delta_ys, visible='legendonly')

    # Create plot figure
    fig = go.Figure(data=[
        simulation_trace,
        model_trace,
        simulation_z_trace,
        model_z_trace,
        simulation_delta_trace,
        model_delta_trace,
    ])

    # Add mean v-line
    fig.add_vline(x=simulation_mean, line_width=3, line_dash="dash", line_color="orange", annotation_text='Valor esperado')

    # Customize plot
    fig.update_layout(barmode="group", title={
        'text': f"Modelo x Simulação: n={n}; s={s}; p={p}",
        'xanchor': 'center',
        'x': 0.5
        },
        template="plotly_dark",
                      xaxis_title="Tempo (k)",
                      yaxis_title="Densidade de Probabilidade")
    fig.show()

    
compare_model_and_sim(50, 25, 0.005)
compare_model_and_sim(100, 25, 0.007)
compare_model_and_sim(100, 40, 0.01)


### Análise da Simulação Completa

Vamos agora sair da validação e solução analítica e partir para cenários mais reais, com tempo de reparo das máquinas e um crescimento da chance de quebra ao longo do tempo. Além disso, procuraremos encontrar quantidades de máquinas reservas necessárias para garantir um tempo mínimo de operação do sistema.

In [12]:

n = 100
tr = 36
p = 0.001
beta = 0.0001

st_low = 0
st_high = 55


sim_data = load_or_generate_simulation(10000, n, p, st_high, tr, beta, force_recreation=False)
sim_df = pd.DataFrame({
    'y': sim_data['results'],
    'z': sim_data['results_z'],
})


def sim_s_search(n: int, p: float, tr: int, beta: float, starting_s: int = 0, ending_s: int = 100, quantile: float = 0.01, target: int = 10000, trials: int = 10000, **kwargs):
    simulations = {}
    cur_simulation = None

    s_low = starting_s
    s_high = ending_s
    
    while s_low < s_high:
        cur_s = (s_high + s_low) // 2
        print(f"Trying {cur_s}")
        cur_simulation = load_or_generate_simulation(trials, n, p, cur_s, tr, beta, **kwargs)
        simulations[cur_s] = cur_simulation
        quantile_val = np.quantile(cur_simulation['results'], quantile)
        print(f"{cur_s} quantile: {quantile_val}")
        
        if quantile_val < target:
            s_low = cur_s + 1
        else:
            s_high = cur_s
    
    return s_low, simulations[s_low]

In [17]:
# sim_s_search(100, 0.001, 36, 0.0001, 40, 60, trials=1000)
sim_s_search(100, 0.001, 38, 0.0001, 40, 80, trials=2000)

Trying 60
60 quantile: 50000.0
Trying 50
50 quantile: 241.98000000000002
Trying 55
55 quantile: 2482.29
Trying 58
58 quantile: 43210.83
Trying 57
57 quantile: 18078.84
Trying 56
56 quantile: 4969.070000000001


(57,
 {'trial_count': 2000,
  'configs': {'n': 100,
   'p0': 0.001,
   's0': 57,
   'tr': 38,
   'beta': 0.0001,
   'seed': 1670854350003},
  'results': [50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   18202,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,
   50000,